In [ ]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import declarative_base, sessionmaker, relationship
from sqlalchemy.schema import Table, Column, ForeignKey
from sqlalchemy.types import Integer, Text
from sqlalchemy.sql import select, insert, update, delete, join, func
import sqlalchemy

In [254]:
# reset
sess.rollback()
sess.close()
engine.dispose()
base.metadata.clear()
base.registry.dispose()

2023-09-08 07:19:32,534 INFO sqlalchemy.engine.Engine ROLLBACK


In [255]:
# base
engine = create_engine('sqlite:///:memory:', echo = True)
base = declarative_base()
Session = sessionmaker(engine)
sess= Session()

In [256]:
"""
- 23.09.07 과제 - 

ORM을 이용하여 SNS의 게시글 추가/수정/삭제 에 대한 로직을 완성하시오

"""

# Artist 1:N Ablums (관계)
# Post N:M HashTag => 관계를 정의하고 있는 별도의 테이블을 써야함.
# Post               Tags              Hashtag
# tags-------------post/hashtag----------posts
# Posts.tags[0].hashtag -------------> Hashtag

class Post(base):
    __tablename__ = 'POST'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    content = Column('CONTENT', Text)
    # tags는 Tags라는 Table과 관계를 갖고 있고, post를 통해 연결됨
    tags = relationship('Tags', back_populates='post', uselist=True)

    # tag 추가
    def addTags(self, tags):
        """
        IDEA) Tag 추가 함수:
        로직은 tag를 반복시켜서 Hashtag Table에 name과 매칭이 되면
        값을 추가하고, 매칭 값이 없으면 Tag를 추가한다.

        이후 추가된 Tag의 cnt값을 하나 증가시킨다.
        """
        tagList = []
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() > 0:
                tagList.append(rst.one())
            else:
                newTag = Hashtag(name = tag)
                sess.add(newTag)
                sess.commit()
                tagList.append(newTag)
        for tag in tagList:
            sess.add(Tags(fk1 = self.pk, fk2 = tag.pk))
            sess.commit()
            tag.cnt += 1

    # tag 삭제    
    def deleteTags(self, tags):
        """
        IDEA) Tag 삭제 함수:
        로직은 특정한 pk를 갖는 tag를 찾아서 (filter? or macthing?)
        삭제한다. (tag가 포함된 모든 post를 삭제하면 안되고, 글을 특정하는 pk와 태그를 매칭 시킨 후
        해당 row를 삭제한다) 이후 삭제된 Tag의 cnt값을 하나 감소시킨다.
        """        
        del_tagList = []
        for tag in tags:
            rst = sess.query(Hashtag).filter(Hashtag.name == tag)
            if rst.count() == 0:
                continue
            else:
                # 삭제할 tag를 뽑는다.
                deltag = sess.query(Hashtag).filter(Hashtag.name == tag).one()          
                sess.commit()
                del_tagList.append(deltag)
        for tag in del_tagList:
            # 삭제할 태그의 pk값 (Tags와 Hashtags의 상속관계를 사용해서 M:M 테이블을 매칭 시키는 테이블의 인자 삭제)
            sess.query(Tags).filter(Tags.fk1 == self.pk, Tags.fk2 == tag.pk).delete()
            sess.commit()
            # 삭제된 태그의 값을 하나 줄임.
            tag.cnt -= 1

    # tag 수정    
    def modifiedTags(self, tag1, tag2):
        """
        IDEA) Tag 수정 함수: 
        바꾸고자 하는 post의 pk와 Tag를 매칭 시킨 후 (pk = my.pk, Tag = my.tag),
        Tag의 값을 바꾼다. (Tag = '어쩌구', sess.commit())
        특정 Tag가 하나 사라지고, 특정 Tag가 하나 증가하였으므로
        추가된 Tag는 cnt +=1, 사라진 Tag는 cnt -=1을 취한다. (+-= 0)
        > 그냥 기존에 만들어둔 deleteTag와 addTag를 사용한다.

        tag1: 수정 전 태그
        tag2: 수정 후 태그
        """        
        self.deleteTags(tag1)
        self.addTags(tag2)
        

    def __repr__(self):
        return f'{self.pk}, {len(self.tags)}'
    
class Hashtag(base):
    __tablename__ = 'HASHTAG'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    name = Column('NAME', Text)
    cnt = Column('CNT', Integer, server_default='0')
    posts = relationship('Tags', back_populates='hashtag', uselist=True)

    def __repr__(self):
        return f'{self.pk}, {self.name}, {len(self.posts)}'    

class Tags(base):
    __tablename__ = 'TAGS'
    __table_args__ = {'extend_existing': True}
    pk = Column('PK', Integer, primary_key= True)
    fk1 = Column('FK1', Integer, None, ForeignKey('POST.PK'))
    fk2 = Column('FK2', Integer, None, ForeignKey('HASHTAG.PK'))
    post = relationship('Post', back_populates='tags')
    hashtag = relationship('Hashtag', back_populates='posts')

In [257]:
# engine에 반영
base.metadata.create_all(engine)

2023-09-08 07:19:39,399 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:39,400 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("POST")
2023-09-08 07:19:39,401 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 07:19:39,402 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("POST")
2023-09-08 07:19:39,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 07:19:39,404 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("HASHTAG")
2023-09-08 07:19:39,404 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 07:19:39,406 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("HASHTAG")
2023-09-08 07:19:39,408 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 07:19:39,410 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("TAGS")
2023-09-08 07:19:39,410 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 07:19:39,411 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("TAGS")
2023-09-08 07:19:39,412 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-09-08 

In [258]:
# 테그 추가 예시
post1 = Post(content = '내용1')
sess.add(post1)
post2 = Post(content = '내용2')
sess.add(post2)
sess.commit()

tag1 = ['해시태그1', '해시태그2']
post1.addTags(tag1)

tag2 = ['해시태그1', '해시태그3']
post2.addTags(tag2)

sess.commit()

2023-09-08 07:19:41,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:41,013 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?) RETURNING "PK"
2023-09-08 07:19:41,014 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('내용1',)
2023-09-08 07:19:41,014 INFO sqlalchemy.engine.Engine INSERT INTO "POST" ("CONTENT") VALUES (?) RETURNING "PK"
2023-09-08 07:19:41,015 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('내용2',)
2023-09-08 07:19:41,017 INFO sqlalchemy.engine.Engine COMMIT
2023-09-08 07:19:41,020 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:41,022 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-08 07:19:41,023 INFO sqlalchemy.engine.Engine [generated i

In [259]:
# 확인
sess.query(Hashtag).all()
"""
pk     hashtag    cnt
1     해시태그1     2
2     해시태그2     1
3     해시태그3     1
"""

2023-09-08 07:19:43,045 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:43,045 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG"
2023-09-08 07:19:43,060 INFO sqlalchemy.engine.Engine [generated in 0.00075s] ()


'\npk     hashtag    cnt\n1     해시태그1     2\n2     해시태그2     1\n3     해시태그3     1\n'

In [260]:
# 테그 제거 예시
de1_tag1 = ['해시태그1']
post1.deleteTags(de1_tag1)
sess.commit()

de1_tag2 = ['해시태그1', '해시태그3']
post2.deleteTags(de1_tag2)
sess.commit()

2023-09-08 07:19:44,652 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-08 07:19:44,652 INFO sqlalchemy.engine.Engine [cached since 3.631s ago] ('해시태그1',)
2023-09-08 07:19:44,654 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-08 07:19:44,656 INFO sqlalchemy.engine.Engine [cached since 3.577s ago] ('해시태그1',)
2023-09-08 07:19:44,657 INFO sqlalchemy.engine.Engine COMMIT
2023-09-08 07:19:44,659 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:44,660 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2023-09-08 07:19:44,661 INFO sqlalchemy.engine.Engine [cached since 3.6

In [261]:
# 확인
sess.query(Hashtag).all()
"""
pk     hashtag    cnt
1     해시태그1     0
2     해시태그2     1
3     해시태그3     0
"""

2023-09-08 07:19:47,132 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:47,133 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG"
2023-09-08 07:19:47,133 INFO sqlalchemy.engine.Engine [cached since 4.075s ago] ()


'\npk     hashtag    cnt\n1     해시태그1     0\n2     해시태그2     1\n3     해시태그3     0\n'

In [262]:
# 테그 수정 예시
bf_tag1 = ['해시태그2']
af_tag1 = ['해시태그4']

post1.modifiedTags(bf_tag1, af_tag1)
sess.commit()

2023-09-08 07:19:51,908 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?) AS anon_1
2023-09-08 07:19:51,909 INFO sqlalchemy.engine.Engine [cached since 10.89s ago] ('해시태그2',)
2023-09-08 07:19:51,911 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG" 
WHERE "HASHTAG"."NAME" = ?
2023-09-08 07:19:51,913 INFO sqlalchemy.engine.Engine [cached since 10.83s ago] ('해시태그2',)
2023-09-08 07:19:51,914 INFO sqlalchemy.engine.Engine COMMIT
2023-09-08 07:19:51,915 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:51,916 INFO sqlalchemy.engine.Engine SELECT "POST"."PK" AS "POST_PK", "POST"."CONTENT" AS "POST_CONTENT" 
FROM "POST" 
WHERE "POST"."PK" = ?
2023-09-08 07:19:51,917 INFO sqlalchemy.engine.Engine [cached since 10.

In [263]:
# 확인
sess.query(Hashtag).all()

"""
pk     hashtag    cnt
1     해시태그1     0
2     해시태그2     0
3     해시태그3     0
4     해시태그4     1
"""

2023-09-08 07:19:54,069 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-09-08 07:19:54,070 INFO sqlalchemy.engine.Engine SELECT "HASHTAG"."PK" AS "HASHTAG_PK", "HASHTAG"."NAME" AS "HASHTAG_NAME", "HASHTAG"."CNT" AS "HASHTAG_CNT" 
FROM "HASHTAG"
2023-09-08 07:19:54,071 INFO sqlalchemy.engine.Engine [cached since 11.01s ago] ()
2023-09-08 07:19:54,074 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2" 
FROM "TAGS" 
WHERE ? = "TAGS"."FK2"
2023-09-08 07:19:54,075 INFO sqlalchemy.engine.Engine [generated in 0.00151s] (1,)
2023-09-08 07:19:54,076 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2" 
FROM "TAGS" 
WHERE ? = "TAGS"."FK2"
2023-09-08 07:19:54,077 INFO sqlalchemy.engine.Engine [cached since 0.00401s ago] (2,)
2023-09-08 07:19:54,079 INFO sqlalchemy.engine.Engine SELECT "TAGS"."PK" AS "TAGS_PK", "TAGS"."FK1" AS "TAGS_FK1", "TAGS"."FK2" AS "TAGS_FK2

[1, 해시태그1, 0, 2, 해시태그2, 0, 3, 해시태그3, 0, 4, 해시태그4, 1]